In [102]:
import json
import pennylane as qml
import pennylane.numpy as np
from pennylane import qaoa 

def finite_gs_energy(L, J, g, return_psi=False):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.
    """
    if L >= 20:
        warnings.warn("Large L: Exact diagonalization might take a long time!")
    # get single site operaors
    sx = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]))
    sz = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]))
    id = sparse.csr_matrix(np.eye(2))
    sx_list = []  # sx_list[i] = kron([id, id, ..., id, sx, id, .... id])
    sz_list = []
    for i_site in range(L):
        x_ops = [id] * L
        z_ops = [id] * L
        x_ops[i_site] = sx
        z_ops[i_site] = sz
        X = x_ops[0]
        Z = z_ops[0]
        for j in range(1, L):
            X = sparse.kron(X, x_ops[j], 'csr')
            Z = sparse.kron(Z, z_ops[j], 'csr')
        sx_list.append(X)
        sz_list.append(Z)
    H_zz = sparse.csr_matrix((2**L, 2**L))
    H_x = sparse.csr_matrix((2**L, 2**L))
    for i in range(L - 1):
        H_zz = H_zz + sz_list[i] * sz_list[(i + 1) % L]
    for i in range(L):
        H_x = H_x + sx_list[i]
    H = -J * H_zz - g * H_x
    E, V = eigsh(H, k=1, which='SA', return_eigenvectors=True, ncv=20)
    if return_psi:
        return E[0], V[:, 0]
    return E[0]

class AbsMagnetization(qml.measurements.StateMeasurement):
    """Custom measurement for calculating the absolute magnetization."""

    def process_state(self, state):
        """Process the quantum state to compute the absolute magnetization.

        Args:
            state (array): Complex-valued quantum state vector.

        Returns:
            float: Absolute magnetization.
        """

        # Get the number of qubits
        num_qubits = int(np.log2(len(state)))

        # Initialize the absolute magnetization
        abs_magnetization = 0

        # Loop over all possible spin configurations
        for sigma in range(2 ** num_qubits):
            sigma_binary = bin(sigma)[2:].zfill(num_qubits)  # Convert to binary
            sigma_state = np.array([int(bit) for bit in sigma_binary])  # Convert to array

            # Compute the expectation value of the Pauli Z operator for this spin configuration
            expectation = np.abs(np.vdot(state, np.kron([1], [1] * (2 ** num_qubits - 1)) @ np.kron([1], np.diag([(-1) ** bit for bit in sigma_state])) @ state)) ** 2

            # Add the contribution to the absolute magnetization
            abs_magnetization += expectation

        return abs_magnetization
def tfim_ground_state(num_qubits, h):
    g, v = finite_gs_energy(num_qubits, -1,-h, True)
    return v
def magnetization(num_qubits, h):
    ground_state_vector = tfim_ground_state(num_qubits, h)
    # Put your code here #
    ground_state_vector = np.array(ground_state_vector)

    # Calculate the total magnetization
    total_magnetization = np.sum(ground_state_vector)

    # Calculate the average magnetization per site
    average_magnetization = total_magnetization / len(ground_state_vector)
    return average_magnetization

def critical_point_estimate(mags, h_values):
    differences = [np.abs(mags[i] - mags[i + 1]) for i in range(len(mags) - 1)]
    ind = np.argmax(np.array(differences))

    h_c = np.mean([h_values[ind], h_values[ind + 1]])
    return h_c


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    num_qubits = json.loads(test_case_input)
    h_values = np.arange(0.2, 1.1, 0.005)
    mags = []

    for h in h_values:
        mags.append(magnetization(num_qubits, h) / num_qubits)
    print (mags)
    output = critical_point_estimate(np.array(mags), h_values)
    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    print (solution_output, expected_output)
    assert np.isclose(solution_output, expected_output, rtol=5e-3)


# These are the public test cases
test_cases = [
    ('2', '0.3535'),
    # ('5', '0.6735'),
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '2'...
[tensor(-0.1401572, requires_grad=True), tensor(-0.13926669, requires_grad=True), tensor(0.13837986, requires_grad=True), tensor(0.13749685, requires_grad=True), tensor(-0.13661778, requires_grad=True), tensor(0.13574279, requires_grad=True), tensor(-0.13487198, requires_grad=True), tensor(-0.13400546, requires_grad=True), tensor(0.13314336, requires_grad=True), tensor(-0.13228576, requires_grad=True), tensor(0.13143278, requires_grad=True), tensor(-0.13058449, requires_grad=True), tensor(0.129741, requires_grad=True), tensor(0.12890238, requires_grad=True), tensor(-0.12806872, requires_grad=True), tensor(-0.12724008, requires_grad=True), tensor(-0.12641655, requires_grad=True), tensor(-0.12559819, requires_grad=True), tensor(-0.12478505, requires_grad=True), tensor(-0.12397721, requires_grad=True), tensor(0.1231747, requires_grad=True), tensor(-0.12237759, requires_grad=True), tensor(-0.12158592, requires_grad=True), tensor(-0.12079972, requires_g

AssertionError: 

In [73]:
1.0625000000000009/3

0.35416666666666696

In [91]:
ch = [qml.PauliZ(0)@qml.PauliZ(1), qml.PauliX(1) ]
coeff = [-1, -h]

cost_hamiltonian = qml.Hamiltonian(coeff[::-1], ch[::-1])
cost_hamiltonian = np.matrix(qml.matrix(cost_hamiltonian))
print (cost_hamiltonian)
N = 2
B = 0.2
J = 1

# Initialize the Hamiltonian matrix
H = np.zeros((2**N, 2**N))

# Loop over all possible spin configurations
for i in range(2**N):
    spin_config = np.array([int(x) for x in bin(i)[2:].zfill(N)])  # Convert integer to binary representation
    
    # Calculate the energy contribution from nearest neighbor interactions
    for j in range(N):
        H[i, i] += -B * spin_config[j]  # External magnetic field term
        H[i, i] += -J * spin_config[j] * spin_config[(j+1) % N]  # Interaction term
        
        
cost_hamiltonian = np.array(H)
print(cost_hamiltonian)

eigenvalues, eigenvectors = np.linalg.eigh(cost_hamiltonian)
print (eigenvalues, eigenvectors)
ground_state_index = np.argmin(eigenvalues)
ground_state_energy = eigenvalues[ground_state_index]
ground_state_vector = eigenvectors[ground_state_index]
ground_state_vector = ground_state_vector[:4]

gsv = np.array(ground_state_vector)[0]
gsv


[[-1. +0.j -0.2+0.j  0. +0.j  0. +0.j]
 [-0.2+0.j  1. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  1. +0.j -0.2+0.j]
 [ 0. +0.j  0. +0.j -0.2+0.j -1. +0.j]]
[[ 0.   0.   0.   0. ]
 [ 0.  -0.2  0.   0. ]
 [ 0.   0.  -0.2  0. ]
 [ 0.   0.   0.  -2.4]]
[-2.4 -0.2 -0.2  0. ] [[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


tensor(0., requires_grad=True)

In [96]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import eigsh
import warnings
import scipy.integrate


def finite_gs_energy(L, J, g, return_psi=False):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.
    """
    if L >= 20:
        warnings.warn("Large L: Exact diagonalization might take a long time!")
    # get single site operaors
    sx = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]))
    sz = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]))
    id = sparse.csr_matrix(np.eye(2))
    sx_list = []  # sx_list[i] = kron([id, id, ..., id, sx, id, .... id])
    sz_list = []
    for i_site in range(L):
        x_ops = [id] * L
        z_ops = [id] * L
        x_ops[i_site] = sx
        z_ops[i_site] = sz
        X = x_ops[0]
        Z = z_ops[0]
        for j in range(1, L):
            X = sparse.kron(X, x_ops[j], 'csr')
            Z = sparse.kron(Z, z_ops[j], 'csr')
        sx_list.append(X)
        sz_list.append(Z)
    H_zz = sparse.csr_matrix((2**L, 2**L))
    H_x = sparse.csr_matrix((2**L, 2**L))
    for i in range(L - 1):
        H_zz = H_zz + sz_list[i] * sz_list[(i + 1) % L]
    for i in range(L):
        H_x = H_x + sx_list[i]
    H = -J * H_zz - g * H_x
    E, V = eigsh(H, k=1, which='SA', return_eigenvectors=True, ncv=20)
    print (V)
    if return_psi:
        return E[0], V[:, 0]
    return E[0]

finite_gs_energy(2, -1, -1, True)

[[-0.37174803]
 [ 0.60150096]
 [ 0.60150096]
 [-0.37174803]]


(-2.23606797749979,
 array([-0.37174803,  0.60150096,  0.60150096, -0.37174803]))